In [10]:
!pip install protobuf


[notice] A new release of pip is available: 23.1.2 -> 24.1
[notice] To update, run: C:\Users\Lucia\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [13]:
!pip install mediapipe opencv-python

!pip install -q mediapipe


[notice] A new release of pip is available: 23.1.2 -> 24.1
[notice] To update, run: C:\Users\Lucia\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 24.1
[notice] To update, run: C:\Users\Lucia\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from matplotlib import pyplot as plt
import imghdr
import os

In [ ]:
data_dir = 'data' 
image_exts = ['jpeg','jpg', 'bmp', 'png']

In [ ]:
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('data')

In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:
batch = data_iterator.next()

In [ ]:
batch[1]

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

In [1]:
# Inicializar Mediapipe
mp_face_mesh = mp.solutions.face_mesh
#face_mesh = mp_face_mesh.FaceMesh()
mp_drawing = mp.solutions.drawing_utils
#mp_drawing_styles = mp.solutions.drawing_styles

with mp_face_mesh.FaceMesh(
    static_image_mode = True,
    max_num_faces=1,
    min_detection_confidence=0.5) as face_mesh:

    #Cargar la imagen
    image_path = r"C:\Users\Lucia\Downloads\golden_ratio_women.jpg"
    image = cv2.imread(image_path)

    if image is None:
        print(f"Error al cargar la imagen desde {image_path}. Verifica la ruta y asegúrate de que el archivo exista.")
    else:
        # Convertir la imagen a RGB (Mediapipe usa RGB)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Procesar la imagen con Mediapipe
        results = face_mesh.process(image_rgb)
        
        # Convertir de nuevo a BGR para mostrar usando OpenCV
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

        print("Face Landmarks: ", results.multi_face_landmarks)

        if results.multi_face_landmarks is not None:
            for face_landmarks in results.multi_face_landmarks:
                mp_drawing.draw_landmarks(
                    image=image_bgr,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1),
                    connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=1, circle_radius=1))
                
                # Obtener coordenadas de puntos específicos
                landmarks = face_landmarks.landmark
                h, w, _ = image.shape
                
                # Puntos específicos (ajustar los índices si es necesario)
                points = [
                    (landmarks[33].x * w, landmarks[33].y * h),  # Ojo izquierdo
                    (landmarks[263].x * w, landmarks[263].y * h), # Ojo derecho
                    (landmarks[1].x * w, landmarks[1].y * h),     # Nariz
                    (landmarks[13].x * w, landmarks[152].y * h),   # Mentón
                ]
 
                # Convertir a enteros
                points = [(int(x), int(y)) for x, y in points]
 
                x1, y1 = points[0]  # Ojo izquierdo
                x2, y2 = points[1]  # Ojo derecho
                x3, y3 = points[2]  # Nariz
                x4, y4 = points[3]  # Mentón

                eye_distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
                eye_to_nose = np.sqrt((x3 - x1)**2 + (y3 - y1)**2)
                nose_to_chin = np.sqrt((x4 - x3)**2 + (y4 - y3)**2)

                # Calcular proporciones aureas
                proportion_eye_nose = eye_to_nose / eye_distance
                proportion_nose_chin = nose_to_chin / eye_distance

                if(proportion_eye_nose >= 1.618 and proportion_eye_nose <=1.620):
                    if(proportion_nose_chin>= 1.618 and proportion_nose_chin <=1.620):
                        print("Cumple ocn la proporción aurea")
                
                # Mostrar resultados
                print(f'Proporción ojo-nariz: {proportion_eye_nose}')
                print(f'Proporción nariz-mentón: {proportion_nose_chin}')

                print(eye_distance, eye_to_nose, nose_to_chin)
                
                # Dibujar los puntos y las líneas
                cv2.circle(image_bgr, (x1, y1), 5, (0, 0, 255), -1)
                cv2.circle(image_bgr, (x2, y2), 5, (0, 0, 255), -1)
                cv2.circle(image_bgr, (x3, y3), 5, (0, 0, 255), -1)
                cv2.circle(image_bgr, (x4, y4), 5, (0, 0, 255), -1)
                cv2.line(image_bgr, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.line(image_bgr, (x1, y1), (x3, y3), (255, 0, 0), 2)
                cv2.line(image_bgr, (x3, y3), (x4, y4), (255, 0, 0), 2)

        # Mostrar la imagen
        cv2.imshow('Face Mesh', image_bgr)
        cv2.waitKey(0)
    cv2.destroyAllWindows()



C:\Users\Lucia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Face Landmarks:  [landmark {
  x: 0.494406492
  y: 0.649199188
  z: -0.055725418
}
landmark {
  x: 0.48708567
  y: 0.588065445
  z: -0.144405
}
landmark {
  x: 0.490828395
  y: 0.60497123
  z: -0.0672051087
}
landmark {
  x: 0.464974344
  y: 0.519270718
  z: -0.122048154
}
landmark {
  x: 0.486131817
  y: 0.567666531
  z: -0.158073395
}
landmark {
  x: 0.486257
  y: 0.53968823
  z: -0.153048739
}
landmark {
  x: 0.488174379
  y: 0.470367193
  z: -0.0968193486
}
landmark {
  x: 0.301629514
  y: 0.453373194
  z: 0.0176380686
}
landmark {
  x: 0.48821488
  y: 0.419983983
  z: -0.0911011621
}
landmark {
  x: 0.487634122
  y: 0.392792195
  z: -0.105134897
}
landmark {
  x: 0.488820493
  y: 0.281541437
  z: -0.104525954
}
landmark {
  x: 0.495252877
  y: 0.658723354
  z: -0.0502742976
}
landmark {
  x: 0.496079713
  y: 0.664624572
  z: -0.039968159
}
landmark {
  x: 0.496782899
  y: 0.665624738
  z: -0.0269547757
}
landmark {
  x: 0.498051643
  y: 0.672444
  z: -0.0228288788
}
landmark {
  x